# Homework 2

2011-11989 산업공학과 조재민

In [273]:
%matplotlib inline
import numpy as np
import sympy
import scipy
from scipy.ndimage import imread
import matplotlib.pyplot as plt

# 1.(30pts) Write a code for Algorithm 1.2, 1.3, 1.5 in python and test your code.

## 1.2. Gaussian Elimination with Pivoting

In [274]:
def gaussian_elimination_with_pivoting(A):
    m, n = A.shape
    P = np.eye(m, m)
    for k in range(m-1):
        r = np.argmax(np.abs(A[k:, k])) # maximum row index
        q = r+k
        P[[k, q]] = P[[q, k]]
        A[[k, q]] = A[[q, k]]
        if not A[k, k] == 0:
            A[k+1:, k] = A[k+1:, k] / A[k, k]
            A[k+1:, k+1:] = A[k+1:, k+1:] - A[k+1:, k] * A[k, k+1:]
    L = np.eye(m, m) + np.tril(A, -1)
    U = np.triu(A)
    return L, U, P    

A = np.array([
    [ 1,  0,  0,  0,  1],
    [-1,  1,  0,  0,  1],
    [-1, -1,  1,  0,  1],
    [-1, -1, -1,  1,  1],
    [-1, -1, -1, -1, 1]])
L, U, P = gaussian_elimination_with_pivoting(A)
print(f'PA = LU')
print(f'{P}\n{A}\n=\n{L}\n{U}')

PA = LU
[[ 1.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.]
 [ 0.  0.  0.  1.  0.]
 [ 0.  0.  0.  0.  1.]]
[[ 1  0  0  0  1]
 [-1  1  0  0  2]
 [-1 -1  1  0  4]
 [-1 -1 -1  1  8]
 [-1 -1 -1 -1 16]]
=
[[ 1.  0.  0.  0.  0.]
 [-1.  1.  0.  0.  0.]
 [-1. -1.  1.  0.  0.]
 [-1. -1. -1.  1.  0.]
 [-1. -1. -1. -1.  1.]]
[[ 1  0  0  0  1]
 [ 0  1  0  0  2]
 [ 0  0  1  0  4]
 [ 0  0  0  1  8]
 [ 0  0  0  0 16]]


## 1.3. Solve $Ax = b$ by using $A = LU$.

In [ ]:
def plusol(A, b):
    L, U, P = gaussian_elimination_with_pivoting(A)
    m, n = A.shape
    
    # Forward Elimination to solve Ly = Pb = c
    y = np.zeros([m])
    c = P @ b
    for j in range(m):
        y[j] = c[j] - L[j, :j-1] @ y[:j-1]
        
    # Back substitution to solve Ux = y
    x = np.zeros([n])
    for j in reversed(range(n)):
        if j == n-1:
            x[j] = y[j] / U[j,j]
        else:
            x[j] = (y[j] - U[j, j+1:]@x[j+1:]) / U[j,j]
    
    return x

In [ ]:
A = np.array([
    [1,  1,  2],
    [0, -7, -7],
    [0,  0,  1]])
b=np.array(
    [5, -21, 1])
plusol(A, b)

## 1.5. Cholesky factorization.

In [ ]:
def cholesky(A):
    m, n = A.shape
    for k in range(m):
        assert A[k, k] >= 0, "Matrix should be PD"
        A[k,k] = np.sqrt(A[k,k])
        A[k+1:, k] = A[k+1:m, k] / A[k,k]
        for j in range(k+1, m+1):
            A[j:, j] = A[j:, j] - A[j:, k] @ A[j, k]
        A[k, k+1:] = np.zeros([m-k])
    R = A.transpose()
    return R

# 2. (30pts) Write a code for Algorithm 2.1, Example 2.3 and 2.5 in python and test your code.

## 2.1. Particular solution of $Ax=b$

In [ ]:
def partic_sol(A, b):
    m, n = A.shape
    R, pivcol = sympy.Matrix(np.append(A, b.reshape(m,1), axis=1)).rref()
    R = np.array(R.tolist()).astype(np.float32)
    pivcol = np.array(pivcol)
    r = len(pivcol)
    if max(pivcol) == n:
        x = np.array([])
    else:
        x = np.zeros([n])
        d = R[:, n]
        x[pivcol] = d[:r]
    return x

In [ ]:
A = np.array([[1, 1, 2, -1],
              [4, 3, 6, -5],
              [-1, -2, -4, 0]])
b = np.array([-1, -4, 1])
partic_sol(A ,b)

## Example 2.3.

In [ ]:
def colbasis(A):
    """Basis for the column space"""
    R, pivcol = sympy.Matrix(A).rref()
    R = np.array(R.tolist()).astype(np.float32)
    pivcol = np.array(pivcol)
    C = A[:, pivcol]
    return C

In [ ]:
colbasis(A)

In [ ]:
def rowbasis(A):
    """Basis for the row space"""
    R, pivcol = sympy.Matrix(A).rref()
    R = np.array(R.tolist()).astype(np.float32)
    pivcol = np.array(pivcol)
    r = len(pivcol)
    B = R[:r, :].transpose()
    return B

In [ ]:
rowbasis(A)

In [ ]:
colbasis(A.transpose())

In [ ]:
def nullbasis(A):
    """Basis for the row space"""
    R, pivcol = sympy.Matrix(A).rref()
    R = np.array(R.tolist()).astype(np.float32)
    pivcol = np.array(pivcol)
    m, n = A.shape
    r = len(pivcol)
    freecol = np.arange(n)
    freecol = np.delete(freecol, pivcol)
    N = np.zeros([n, n-r])
    N[freecol, :] = np.eye(n-r)
    N[pivcol, :] = -R[:r, freecol]
    return N

In [ ]:
nullbasis(A)

In [ ]:
A@nullbasis(A)

In [ ]:
def leftnull(A):
    m, n = A.shape
    I = np.eye(m)
    RE, pivcol = sympy.Matrix(np.append(A, I, axis=1)).rref()
    RE = np.array(RE.tolist()).astype(np.float32)
    pivcol = np.array(pivcol)
    r = np.sum(pivcol+1 <= n)
    E = RE[:, n:m+n+1]
    LN = E[r:m+1, :].transpose()
    return LN

In [ ]:
leftnull(A)

In [ ]:
A.transpose()@leftnull(A)

## Example 2.5.

In [ ]:
A = np.array([[1, 1, 2, -1],
              [4, 3, 6, -5],
              [-1, -2, -4, 0]])
b = np.array([-1, -4, 1])

In [ ]:
P, L, U = scipy.linalg.lu(A)

In [ ]:
L

In [ ]:
U

In [ ]:
P

In [ ]:
x1 = partic_sol(A, b)
x1

# 3. (30pts) Write Algorithm 3.1, 3.2, 3.3 in python and test your code.

## 3.1. Classical Gram-Schmidt orthogonalization.

In [ ]:
A = np.array([[1, 1, 2, -1],
              [4, 3, 6, -5],
              [-1, -2, -4, 0]])
b = np.array([-1, -4, 1])

In [ ]:
def clgs(A):
    m, n = A.shape
    V = A
    Q = np.eye(m,n)
    R = np.zeros([n, n])
    for j in range(n):
        for i in range(j-1):
            R[i, j] = np.dot(Q[:, i].transpose(), A[:,j])
            print(R)
            V[:, j] = V[:, j] - R[i, j] * Q[:, i]
        R[j, j] = np.linalg.norm(V[:, j])
        Q[:, j] = V[:, j] // R[j, j]
        print(Q)
    return Q, R

In [ ]:
Q, R = clgs(A)

In [ ]:
Q

In [ ]:
R

In [ ]:
A

In [ ]:
np.linalg.qr(A)

## 3.2. Modified Gram-Schmidt orthogonalization

In [ ]:
def grams(A):
    m, n = A.shape
    Q = A
    R = np.zeros([n, n])
    for i in range(n-1):
        R[i, i] = np.linalg.norm(Q[:, i])
        Q[:, i] = Q[:, i] // R[i, i]
        R[i, i+1:] = Q[:, i].transpose() @ Q[:, i+1:]
        print(Q[:, i])
        print(R[i, i+1:])
        Q[:, i+1:] = Q[:, i+1:] - Q[:, i].transpose() @ R[i, i+1:]
    R[n, n] = np.linalg.norm(Q[:, n])
    Q[:, n] = Q[:, n] // R[n, n]

In [ ]:
grams(A)

## 3.3. Householder QR factorization

In [ ]:
def qrhouse(A):
    m,n = A.shape
    R = A
    V = np.zeros([m, n])
    for k in range(min(m-1, n)):
        x = R[k:m+1, k]
        v = x + np.sign(x[0]) * np.linalg.norm(x) * np.eye(len(x))[0]

        V[k: m+1, k] = v
        R[k:m+1, k:n+1] = R[k:m+1, k:n+1] - (2*v) @ (v.transpose() @ R[k:m+1, k:n+1]) // np.dot(v, v)
    R = np.triu(R[:n, :n])
    return V, R
    
def formQ(V):
    m, n = V.shape
    Q = np.eye(m)
    
    _range = list(range(1, min(m-1, n), 1)).reverse()
    for j in _range:
        v = V[:, j]
        Q = Q- (2*v) @ (v.transpose() @ Q) // np.dot(v, v)
    return Q

In [ ]:
V, R = qrhouse(A)
Q = formQ(V)

# 4. (10pts) Write a code for Figure 5.3 in python and test your code.

## Image processing

In [ ]:
A = imread('./clown.jpg', 'LA')
m, n = A.shape
A.shape

In [ ]:
plt.title('original')
plt.imshow(A, cmap='gray')

In [ ]:
U, S, V = np.linalg.svd(A)
U.shape, S.shape, V.shape

In [ ]:
B = np.zeros([m, n])

In [ ]:
fig, axes = plt.subplots(ncols=(m//20)+1)
fig.set_size_inches(w=30, h=20)
axes = iter(axes)
for i in range(m):
    B = B + S[i] * U[:, i].reshape(m,1) @ V[i, :].reshape(1,n)
    if (i+10) // 20 == 0:
        axis = next(axes)
        axis.set_title(f'i = {i}')
        axis.imshow(B, cmap='gray')